In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import f1_score, classification_report
import gc # 垃圾回收
import matplotlib.pyplot as plt
import seaborn as sns

# 设置 Matplotlib/Seaborn 样式
sns.set(style='whitegrid')
# 设置 Pandas 显示选项
pd.set_option('display.max_columns', None)

In [2]:
print("正在加载特征化训练数据 (train_with_features.csv)...")
# 这可能需要几分钟，请耐心等待
try:
    df_train = pd.read_csv('train_with_features.csv', low_memory=False)
    print(f"数据加载成功！形状: {df_train.shape}")
except FileNotFoundError:
    print("错误：train_with_features.csv 文件未找到。")
    # 如果出错，停止执行
    raise

# -----------------------------------------------------------------
# 关键：恢复数据类型（CSV 丢失了它们）
# -----------------------------------------------------------------
print("正在恢复数据类型...")
# 1. 恢复时间列
df_train['t'] = pd.to_datetime(df_train['t'])

# 2. 恢复 ticker_id 为 'category'
# 这是 LightGBM 高效处理所必需的
df_train['ticker_id'] = df_train['ticker_id'].astype('category')

print("数据类型恢复完毕。")
print(df_train[['t', 'ticker_id', 'class_label']].head())

正在加载特征化训练数据 (train_with_features.csv)...
数据加载成功！形状: (1932, 68736)
正在恢复数据类型...
数据类型恢复完毕。
           t ticker_id class_label
0 2023-04-03         1         NaN
1 2023-04-05         1         NaN
2 2023-04-06         1         NaN
3 2023-04-11         1         NaN
4 2023-04-12         1         NaN


In [3]:
# 1. 定义 5 分类目标映射
# (HH=0, HL=1, LH=2, LL=3, NaN=4)
target_map = {
    'HH': 0,
    'HL': 1,
    'LH': 2,
    'LL': 3
}
# 我们将 'None' (即 NaN) 映射为 4

# 2. 映射
df_train['class_label_encoded'] = df_train['class_label'].map(target_map)
df_train['class_label_encoded'] = df_train['class_label_encoded'].fillna(4)
df_train['class_label_encoded'] = df_train['class_label_encoded'].astype(int)

print("目标变量编码完成。")
print(df_train['class_label_encoded'].value_counts().sort_index())

目标变量编码完成。
class_label_encoded
0      40
1      28
2      18
3      26
4    1820
Name: count, dtype: int64


In [4]:
# 1. 定义目标和元数据
TARGET = 'class_label_encoded'
METADATA_COLS = ['class_label', 'class_label_encoded', 't', 'id', 'train_id']

# 2. 加载 V2 模型的特征重要性
try:
    importance_df = pd.read_csv('feature_importance_v2.csv')
except FileNotFoundError:
    print("错误：'feature_importance_v2.csv' 未找到！")
    print("请先返回 03_Model_v2... 笔记本，运行 CELL 8 以保存该文件。")
    raise

# 3. 定义我们的“精英特征集”
# -----------------------------------------------
NUM_TOP_FEATURES = 300  # 这是一个超参数，我们可以调整它 (例如 200, 300, 500)
# -----------------------------------------------

# 4. 获取 Top N 特征的列表
top_features = importance_df.head(NUM_TOP_FEATURES)['feature'].tolist()

# 5. 确保 'ticker_id' 被包含在内！
# (即使它重要性不高，它对模型结构也至关重要)
if 'ticker_id' not in top_features:
    top_features.append('ticker_id')

# 6. 将此列表设置为我们的最终 FEATURES
FEATURES = top_features

print(f"--- Model v3 特征选择 ---")
print(f"已加载 {len(FEATURES)} 个 Top 特征 (Top {NUM_TOP_FEATURES} + 'ticker_id')。")

--- Model v3 特征选择 ---
已加载 300 个 Top 特征 (Top 300 + 'ticker_id')。


In [5]:
# -----------------------------------------------------------------# 单元格 5：建立时间序列验证框架 (TSV) + 类型修复# -----------------------------------------------------------------
# 1. 确保数据严格按 *原始* 基线的方式排序
print("正在按 't' 和 'ticker_id' 排序数据以进行验证分割...")
# 注意：确保 df_train 在这里仍然是您在 CELL 2 中加载的完整数据
df_train = df_train.sort_values(by=['t', 'ticker_id']).reset_index(drop=True)

# 2. 定义分割点 (与基线完全一致)
split_percentage = 0.8
split_index = int(len(df_train) * split_percentage)

# 3. 分割数据
train_data = df_train.iloc[:split_index]
val_data = df_train.iloc[split_index:]

print(f"总训练样本数: {len(df_train)}")
print(f"新训练集样本数: {len(train_data)}")
print(f"新验证集样本数: {len(val_data)}")
print(f"验证集时间范围: {val_data['t'].min()} 到 {val_data['t'].max()}")

# 4. 创建 X (特征) 和 y (目标)# FEATURES 列表应该已经在您上一个单元格 (CELL 4) 中定义了
X_train = train_data[FEATURES]
y_train = train_data[TARGET]

X_val = val_data[FEATURES]
y_val = val_data[TARGET]

# -----------------------------------------------------------------# 💡 【错误修复】 强制转换数据类型# -----------------------------------------------------------------
print("正在修复 'object' 数据类型...")

# 找出 'ticker_id' 之外的所有特征列
numeric_features = [col for col in FEATURES if col != 'ticker_id']

# 我们使用 .copy() 来避免 Pandas 的 SettingWithCopyWarning
X_train_copy = X_train.copy()
X_val_copy = X_val.copy()

for col in numeric_features:
    X_train_copy[col] = pd.to_numeric(X_train_copy[col], errors='coerce', downcast='float')
    X_val_copy[col] = pd.to_numeric(X_val_copy[col], errors='coerce', downcast='float')

# 替换回原来的变量
X_train = X_train_copy
X_val = X_val_copy

print(f"已将 {len(numeric_features)} 个特征列强制转换为数值类型。")
# -----------------------------------------------------------------# 5. 清理内存del df_train, train_data, val_data, X_train_copy, X_val_copy
gc.collect()

print("时间序列验证集创建完毕！")

正在按 't' 和 'ticker_id' 排序数据以进行验证分割...
总训练样本数: 1932
新训练集样本数: 1545
新验证集样本数: 387
验证集时间范围: 2024-09-20 00:00:00 到 2025-01-31 00:00:00
正在修复 'object' 数据类型...
已将 299 个特征列强制转换为数值类型。
时间序列验证集创建完毕！


In [6]:
# 1. 定义 LightGBM 模型参数 (与 V2 相同)
lgb_params = {
    'objective': 'multiclass',
    'num_class': 5,
    'metric': 'multi_logloss',
    'n_estimators': 500,
    'learning_rate': 0.05,
    'n_jobs': -1,
    'seed': 42,
    'class_weight': 'balanced'
}

# 2. 初始化模型
model_v3 = lgb.LGBMClassifier(**lgb_params)

# 3. 训练模型
print(f"开始训练 Model v3 (使用 Top {len(FEATURES)} 特征)...")
model_v3.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='multi_logloss',
    callbacks=[lgb.early_stopping(50)],
    categorical_feature=['ticker_id'] # 必须指定
)

print("Model v3 训练完成！")

开始训练 Model v3 (使用 Top 300 特征)...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004675 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13692
[LightGBM] [Info] Number of data points in the train set: 1545, number of used features: 276
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[64]	valid_

In [7]:
# 1. 在验证集上进行预测
print("正在验证集上进行预测...")
val_preds = model_v3.predict(X_val)

# 2. 计算 Macro F1-Score
macro_f1_v3 = f1_score(y_val, val_preds, average='macro', zero_division=0)

print("\n========================================")
print(f"📈 Model v3 (Top {NUM_TOP_FEATURES}) Macro F1-Score: {macro_f1_v3:.4f}")
print(f"📈 Model v2 (All Features) Macro F1-Score: 0.2238")
print(f"📉 Baseline Macro F1-Score: 0.1905")
print("========================================")
print(f"🚀 V3 vs V2 性能提升: {macro_f1_v3 - 0.2238:.4f}")
print("========================================")

# 3. 打印详细的分类报告
target_map_full = {'HH': 0, 'HL': 1, 'LH': 2, 'LL': 3, 'None': 4}
target_names = [key for key, val in sorted(target_map_full.items(), key=lambda item: item[1])]
all_labels = [val for key, val in sorted(target_map_full.items(), key=lambda item: item[1])]

print("\n详细分类报告 (Model v3):")
print(classification_report(
    y_val, 
    val_preds, 
    target_names=target_names, 
    labels=all_labels,
    zero_division=0
))

正在验证集上进行预测...

📈 Model v3 (Top 300) Macro F1-Score: 0.1916
📈 Model v2 (All Features) Macro F1-Score: 0.2238
📉 Baseline Macro F1-Score: 0.1905
🚀 V3 vs V2 性能提升: -0.0322

详细分类报告 (Model v3):
              precision    recall  f1-score   support

          HH       0.00      0.00      0.00        10
          HL       0.00      0.00      0.00         9
          LH       0.00      0.00      0.00         4
          LL       0.00      0.00      0.00         5
        None       0.93      0.99      0.96       359

    accuracy                           0.92       387
   macro avg       0.19      0.20      0.19       387
weighted avg       0.86      0.92      0.89       387

